In [40]:
import os
import struct
import pandas as pd

# import taos
import sys
import datetime
import random

# def exitProgram(conn):
#     conn.close()
#     sys.exit()
    
    
# def getconn():
#     conn = taos.connect(host="127.0.0.1", user="root", password="taosdata", config="/etc/taos")
#     return conn
def df2db(df):
    df.to_sql(name='test', con=con, if_exists='append', index=False)

def parse_tdxfile(fname):
    dataSet=[]
    with open(fname,'rb')  as fl:
        buffer=fl.read()  #读取数据到缓存
        size=len(buffer)  
        rowSize=32 #通信达day数据，每32个字节一组数据
        code=os.path.basename(fname).replace('.day','')
        for i in range(0,size,rowSize):  #步长为32遍历buffer
            row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
            row[1]=row[1]/100
            row[2]=row[2]/100
            row[3]=row[3]/100
            row[4]=row[4]/100
            row.pop() #移除最后无意义字段
            row.insert(0,code)
            dataSet.append(row)  
    df = pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
    return df


def parse_folder(folder_name):
    print(folder_name)
    path = folder_name
    files= os.listdir(path)
    for fname in files: #遍历文件夹
        if not os.path.isdir(fname): #判断是否是文件夹，不是文件夹才打开
            fullname = "{0}/{1}".format(folder_name, fname)
            print(fullname)
            df = parse_tdxfile(fullname)
            # TODO save to ?

def main(): 
    folder_name = "/Users/cwj/Downloads/stk-data//sh-sz-lday-2004-20190821"
    fname = folder_name +"/sh000001.day"
    # parse_folder(folder_name)
    df = parse_tdxfile(fname)
    print(df.tail())
    print('- ' * 20)
    print(df.shape)
    print("-end-")

In [41]:
if __name__ == '__main__':
    main()

          code  tradeDate     open     high      low    close        amount  \
7007  sh000001   20190815  2762.34  2815.90  2756.83  2815.80  1.736705e+11   
7008  sh000001   20190816  2817.57  2840.32  2811.80  2823.82  1.861394e+11   
7009  sh000001   20190819  2835.52  2883.10  2829.85  2883.10  2.470922e+11   
7010  sh000001   20190820  2879.08  2892.08  2875.00  2880.00  2.097743e+11   
7011  sh000001   20190821  2875.47  2885.59  2872.56  2880.33  1.831873e+11   

            vol  
7007  152772152  
7008  148582344  
7009  214546668  
7010  189677206  
7011  159019665  
- - - - - - - - - - - - - - - - - - - - 
(7012, 8)
-end-
